In [1]:
from lstm_models import LSTM
from gpu_dataloader import GPUDataset
from torch.utils.data import DataLoader, SubsetRandomSampler
import torch.nn as nn
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler, StandardScaler

# plotting the data
import matplotlib.pyplot as plt
# used for the dataframes
import pandas as pd
from tqdm import tqdm

import random

from utils import get_device


import numpy as np
%matplotlib inline

from utils import get_device_as_string, get_device

/home/chrispy/miniconda3/envs/ml_pytorch/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import math

def get_rmse(actual_values, predicted_values) -> float:
    '''returns the root mean squared error'''
    return math.sqrt(mean_squared_error(actual_values, predicted_values))

def get_mape(actual_values, predicted_values):
    '''returns the mean absolue percentage error'''
    return np.mean(np.abs(actual_values - predicted_values) / np.abs(actual_values) * 100)

def get_mae(actual_values, predicted_values) -> float:
    '''returns the mean absolute error'''
    return mean_absolute_error(actual_values, predicted_values)

In [3]:
batch_size: int = 200
dataset = GPUDataset(small_df=True, batch_size=batch_size)

In [4]:
num_epochs: int = 3
learning_rate: float = 0.001

# number of features
input_size: int = dataset.X.shape[2]
# number of features in hidden state
hidden_size: int = dataset.X.shape[2] * 128
# number of stacked lstm layers
num_layers: int = 1
# number of output classes
num_classes: int = dataset.y.shape[1]

device = get_device()

In [5]:
import wandb
wandb.init(project='Hardware Utilization Prediction')

wandb.config.num_epochs = num_epochs
wandb.config.learning_rate = learning_rate
wandb.config.input_size = input_size
wandb.config.hidden_size = hidden_size
wandb.config.num_layers = num_layers
wandb.config.num_classes = num_classes

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: my-god-its-full-of-stars. Use `wandb login --relogin` to force relogin


In [6]:
lstm = LSTM(num_classes, input_size, hidden_size, num_layers, dataset.X.shape[1])

In [7]:
LOSS: str = 'loss'
RMSE_TRAINING: str = 'root mean squared error (training)'
MAE_TRAINING: str = 'mean absolute error (training)'

wandb.define_metric(LOSS, summary='min')
wandb.define_metric(RMSE_TRAINING, summary='min')
wandb.define_metric(MAE_TRAINING, summary='min')

In [8]:
# lstm = lstm.to(device)
lstm.train()
# log gradients and model parameters
wandb.watch(lstm)

lstm


LSTM(
  (lstm): LSTM(19, 1216, batch_first=True)
  (dropout): Dropout(p=0.25, inplace=False)
  (fc_1): Linear(in_features=1216, out_features=512, bias=True)
  (fc_2): Linear(in_features=512, out_features=256, bias=True)
  (fc_3): Linear(in_features=256, out_features=1, bias=True)
  (relu): LeakyReLU(negative_slope=0.01)
)

In [9]:
class RMSELoss(nn.Module):
    def __init__(self, eps=1e-6):
        super().__init__()
        self.mse = nn.MSELoss()
        self.eps = eps

    def forward(self, y_pred, y):
        loss = torch.sqrt(self.mse(y_pred, y) + self.eps)
        return loss


In [10]:
# mean squared error for regression
# criterion = nn.MSELoss(size_average=5)
# criterion = criterion.to(get_device())
criterion = RMSELoss()
criterion = criterion.to(device)
# optimizer function
optimizer = torch.optim.AdamW(lstm.parameters(), lr=learning_rate)

In [11]:
def log_training_metrics(predictions, labels, loss):
    # logging to wandb
    if get_device_as_string() == 'cuda':
        o = predictions.cpu().detach().numpy()
        l = labels.cpu().detach().numpy()
    else:
        o = predictions.detach().numpy()
        l = labels.detach().numpy()
    rmse = get_rmse(o, l)
    mae = get_mae(o, l)
    log_dict: dict = {
        LOSS: loss.item(),
        RMSE_TRAINING: rmse,
        MAE_TRAINING: mae,
    }
    wandb.log(log_dict)

In [12]:


for epoch in tqdm(range(0, num_epochs), desc="Training Loop"):

    batch_section = [idx for idx in range(len(dataset) // batch_size)]

    while len(batch_section) > 0:
        choice = random.choice(batch_section)
        sampler = SubsetRandomSampler(
            list(range(choice*batch_size, (choice+1)*batch_size)))
        train_loader = DataLoader(
            dataset, batch_size=batch_size, shuffle=False, num_workers=6, sampler=sampler)
        
        # train_loader = tqdm(train_loader)

        for _, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)
            
            optimizer.zero_grad()

            predictions = lstm.forward(inputs)
            predictions = predictions.to(device)

            loss = criterion(predictions, labels)
            loss.backward()
            optimizer.step()
            log_training_metrics(predictions, labels, loss)

        # remove batch section after training on it
        batch_section.remove(choice)
        

    print(f'Epoch: {epoch}, loss: {loss.item():2f}')


Training Loop:   1%|          | 1/100 [00:11<18:11, 11.03s/it]

Epoch: 0, loss: 0.017037


Training Loop:   2%|▏         | 2/100 [00:22<17:58, 11.01s/it]

Epoch: 1, loss: 0.020168


Training Loop:   3%|▎         | 3/100 [00:33<17:50, 11.03s/it]

...

Epoch: 3, loss: 0.031968


Training Loop:   5%|▌         | 5/100 [00:55<17:36, 11.12s/it]

Epoch: 4, loss: 0.018207


Exception ignored in: <function _releaseLock at 0x7f1803697040>
Traceback (most recent call last):
  File "/home/chrispy/miniconda3/envs/ml_pytorch/lib/python3.9/logging/__init__.py", line 227, in _releaseLock
    def _releaseLock():
KeyboardInterrupt: 
Training Loop:   6%|▌         | 6/100 [01:06<17:30, 11.18s/it]

Epoch: 5, loss: 0.004633


In [ ]:
import time

current_time = time.ctime()
current_time

## Save the Model to Disk

In [ ]:
# torch.save(lstm.state_dict(), f'models/epochs-{num_epochs}-{current_time}')

In [ ]:
lstm.eval()

X_df = dataset.X
X_df = X_df.to(device)
# forward pass
prediction = lstm(X_df)
if get_device_as_string() == 'cuda':
    prediction = prediction.cpu().data.numpy()
else:
    prediction = prediction.data.numpy()
prediction = dataset.standard_scaler.fit_transform(prediction)

actual_data = dataset.y.data.numpy()
actual_data = dataset.minmax_scaler.fit_transform(actual_data)

# reverse transformation
prediction = dataset.standard_scaler.inverse_transform(prediction)
actual_data = dataset.minmax_scaler.inverse_transform(actual_data)

label_columns = dataset.get_default_label_columns()
# create dataframes
prediction_df = pd.DataFrame(prediction, columns=label_columns)
actual_data_df = pd.DataFrame(actual_data, columns=label_columns)


## Calculate Root Mean Squared Error

Calculating the RMSE for the overall prediction of the (training) dataset.

In [ ]:
rmse_key: str = 'Root Mean Squared Error (Overall - Training)'
rmse_result = get_rmse(actual_data_df[:], prediction_df[:])
print(f'Test Score: {rmse_result:.2f} RMSE')
wandb.summary[rmse_key] = rmse_result

## Calculate Mean Absolute Error

Calcutlate the MAE for the overall prediction of the (training) dataset.

In [ ]:
mae_key: str = 'Mean Absolute Error (Overall - Training)'
mae_result = mean_absolute_error(actual_data_df[:], prediction_df[:])
print(f'Test Score: {mae_result} MAE')
wandb.summary[mae_key] = mae_result

In [ ]:
def plot_column(actual_values=actual_data_df, predicted_values=prediction_df, column_number: int = 0, rmse_threshold: float = 0.30, is_training: bool = True):

    if len(label_columns) <= column_number:
        print('Out of Prediction Bounds')
        return

    plt.figure(figsize=(25, 15))  # plotting

    column = label_columns[column_number]
    pred_column = f"pred_{column}_{'training' if is_training else 'test'}"

    rmse = get_rmse(actual_values[column], predicted_values[column])
    mae = mean_absolute_error(actual_values[column], predicted_values[column])

    predicted_color = 'green' if rmse < rmse_threshold else 'orange'

    plt.plot(actual_values[column], label=column, color='black')  # actual plot
    plt.plot(predicted_values[column], label='pred_' +
             column, color=predicted_color)  # predicted plot

    plt.title('Time-Series Prediction')
    plt.plot([], [], ' ', label=f'RMSE: {rmse}')
    plt.plot([], [], ' ', label=f'MAE: {mae}')
    plt.legend()
    
    wandb.log({pred_column: wandb.Image(plt)})
    wandb.summary[f'Root Mean Squared Error ({column})'] = rmse
    wandb.summary[f'Mean Absolute Error ({column})'] = mae
    
    plt.show()


## See Predictions on Training Dataset

In [ ]:
plot_column(column_number=0)

In [ ]:
plot_column(column_number=1)

In [ ]:
plot_column(column_number=2)

In [ ]:
plot_column(column_number=3)

In [ ]:
plot_column(column_number=4)

In [ ]:
plot_column(column_number=5)

In [ ]:
plot_column(column_number=6)

## Test Set Analysis

Below, the test set will be loaded and the model evaluated with it to see the actual performance.

In [ ]:
test_dataset = GPUDataset(is_training=False, small_df=True, batch_size=batch_size)

In [ ]:
lstm.eval()

X_df = test_dataset.X
X_df = X_df.to(device)
# forward pass
prediction = lstm(X_df)
if get_device_as_string() == 'cuda':
    prediction = prediction.cpu().data.numpy()
else:
    prediction = prediction.data.numpy()
prediction = test_dataset.standard_scaler.fit_transform(prediction)

actual_data = test_dataset.y.data.numpy()
actual_data = test_dataset.minmax_scaler.fit_transform(actual_data)

# reverse transformation
prediction = test_dataset.standard_scaler.inverse_transform(prediction)
actual_data = test_dataset.minmax_scaler.inverse_transform(actual_data)

label_columns = test_dataset.get_default_label_columns()
# create dataframes
prediction_df = pd.DataFrame(prediction, columns=label_columns)
actual_data_df = pd.DataFrame(actual_data, columns=label_columns)


In [ ]:
rmse_key: str = 'Root Mean Squared Error (Overall - Test)'
rmse_result = get_rmse(actual_data_df[:], prediction_df[:])
print(f'Test Score: {rmse_result:.2f} RMSE')
wandb.summary[rmse_key] = rmse_result

In [ ]:
mae_key: str = 'Mean Absolute Error (Overall - Test)'
mae_result = mean_absolute_error(actual_data_df[:], prediction_df[:])
print(f'Test Score: {mae_result} MAE')
wandb.summary[mae_key] = mae_result

In [ ]:
plot_column(column_number=0)

In [ ]:
plot_column(column_number=1)

In [ ]:
plot_column(column_number=2)